# 第三章：分配角色（角色提示）

- [课程内容](#lesson)
- [练习](#exercises)
- [示例练习场](#example-playground)

## 设置

运行以下设置单元格来加载您的 API 密钥并建立 `get_completion` 辅助函数。

In [ ]:

# 导入 Python 内置的正则表达式库
import re
from openai import OpenAI

# 从 IPython 存储中检索 API_KEY 和 MODEL_NAME 变量
%store -r API_KEY
%store -r MODEL_NAME
%store -r BASE_URL

client = OpenAI(
    api_key=API_KEY,
    base_url=BASE_URL
)

def get_completion(prompt: str, system_prompt=""):
    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    messages.append({"role": "user", "content": prompt})
    
    response = client.chat.completions.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=messages
    )
    return response.choices[0].message.content

---

## 课程内容

延续模型除了您告诉它的内容之外没有其他上下文这一主题，有时**让模型扮演特定角色（包括所有必要的上下文）**是很重要的。这也被称为角色提示。角色上下文越详细越好。

**为模型设定角色可以提高模型的性能**，涵盖从写作到编程再到总结等各个领域。这就像人类有时被告知"像______一样思考"时会得到帮助一样。角色提示还可以改变模型响应的风格、语调和方式。

**注意：** 角色提示可以在系统提示中进行，也可以作为用户消息轮次的一部分。

### 示例

在下面的示例中，我们看到在没有角色提示的情况下，当被要求对滑板运动给出一句话的看法时，模型提供了**直接且无风格化的答案**。

然而，当我们让模型扮演猫的角色时，模型的视角发生了变化，因此**模型的响应语调、风格、内容都适应了新角色**。

**注意：** 您可以使用的一个额外技巧是**为模型提供其目标受众的上下文**。在下面的例子中，我们可以调整提示来告诉模型它应该对谁说话。"你是一只猫"与"你是一只对一群滑板手说话的猫"产生的响应截然不同。

这是系统提示中没有角色提示的提示：

In [ ]:
# 提示
PROMPT = "用一句话，你对滑板运动有什么看法？"

# 打印模型的响应
print(get_completion(PROMPT))

这是同样的用户问题，但有角色提示。

In [ ]:
# 系统提示
SYSTEM_PROMPT = "你是一只猫。"

# 提示
PROMPT = "用一句话，你对滑板运动有什么看法？"

# 打印模型的响应
print(get_completion(PROMPT, SYSTEM_PROMPT))

您可以使用角色提示来让模型模仿某些写作风格、以某种声音说话，或指导其答案的复杂性。**角色提示还可以让模型在执行数学或逻辑任务时表现得更好。**

例如，在下面的示例中，大模型的推理常常是错误的：

In [ ]:
# 提示
PROMPT = "杰克在看安妮。安妮在看乔治。杰克已婚，乔治未婚，不知道安妮是否已婚。有已婚的人在看未婚的人吗？"

# 打印模型的响应
print(get_completion(PROMPT))

现在，如果我们**让模型扮演一个不擅长逻辑推理的人**会怎么样？这将如何改变模型的答案？

事实证明，通过这个新的角色分配，模型虽然得到了正确答案，但是推理过程并不正确。

In [ ]:
# 系统提示
SYSTEM_PROMPT = """你是一个精确的逻辑推理专家。在回答问题时，请：
1. 仔细分析所有给定条件
2. 考虑所有可能的情况（穷举法）
3. 对每种可能性进行逐一验证
4. 得出明确的结论
5. 确保推理过程严密无误，不遗漏任何可能性"""

# 提示
PROMPT = "杰克在看安妮。安妮在看乔治。杰克已婚，乔治未婚，我们不知道安妮是否已婚。有已婚的人在看未婚的人吗？"

# 打印模型的响应
print(get_completion(PROMPT, SYSTEM_PROMPT))

**注意：** 在本课程中您将学到的是，有**许多提示工程技术可以用来获得类似的结果**。您使用哪些技术取决于您个人的偏好！我们鼓励您**尝试找到自己的提示工程风格**。

如果您想在不更改上述任何内容的情况下试验课程提示，请滚动到课程笔记本的最底部访问[**示例练习场**](#example-playground)。

---

## 练习
- [练习 3.1 - 数学纠正](#exercise-31---math-correction)

### 练习 3.1 - 数学纠正
在某些情况下，**模型可能在数学方面有困难**，即使是简单的数学。下面，模型错误地评估数学问题已正确解决，尽管第二步中有明显的算术错误。注意模型在逐步检查时实际上发现了错误，但没有得出整体解决方案错误的结论。

修改 `PROMPT` 和/或 `SYSTEM_PROMPT`，使模型将解决方案评为`错误`解决，而不是正确解决。 


In [36]:
# 系统提示 - 如果您不想使用系统提示，可以将此变量设置为空字符串
SYSTEM_PROMPT = "[填写你的提示词]"

# 提示
PROMPT = """下面的方程解得正确吗？

2x - 3 = 9

2x = 6

x = 3"""

# 获取模型的响应
response = get_completion(PROMPT, SYSTEM_PROMPT)

# 用于评分练习正确性的函数
def grade_exercise(text):
    if "incorrect" in text or "not correct" in text.lower() or "错误" in text or "不正确" in text:
        return True
    else:
        return False

# 打印模型的响应和相应的评分
print(response)
print("\n--------------------------- 评分 ---------------------------")
print("这个练习已正确解决:", grade_exercise(response))

是的，解是正确的。

从方程 2x - 3 = 9 开始，首先将等式两边同时加3，得到：

2x = 12

然后，将等式两边同时除以2，得到：

x = 6

看起来在你的解题过程中，第二步应该是 2x = 12 而不是 2x = 6。因此，正确的解是 x = 6。

--------------------------- 评分 ---------------------------
这个练习已正确解决: False


❓ 如果您需要提示，请运行下面的单元格！

In [ ]:
from hints import exercise_3_1_hint; print(exercise_3_1_hint)

### 恭喜！

如果您已经解决了到目前为止的所有练习，您就可以进入下一章了。愉快的提示工程学习！

---

## 示例练习场

这是一个供您自由试验本课程中展示的提示示例并调整提示以查看如何影响模型响应的区域。

In [ ]:
# 提示
PROMPT = "用一句话，你对滑板运动有什么看法？"

# 打印模型的响应
print(get_completion(PROMPT))

In [ ]:
# 系统提示
SYSTEM_PROMPT = "你是一只猫。"

# 提示
PROMPT = "用一句话，你对滑板运动有什么看法？"

# 打印模型的响应
print(get_completion(PROMPT, SYSTEM_PROMPT))

In [39]:
# 提示
PROMPT = "杰克在看安妮。安妮在看乔治。杰克已婚，乔治未婚，我们不知道安妮是否已婚。有已婚的人在看未婚的人吗？"

# 打印模型的响应
print(get_completion(PROMPT))

根据题目描述，我们可以得出以下信息：

1. 杰克在看安妮。
2. 安妮在看乔治。
3. 杰克已婚。
4. 乔治未婚。
5. 安妮的婚姻状况未知。

我们需要判断是否有已婚的人在看未婚的人。

根据信息，杰克是已婚的，而乔治是未婚的。杰克在看安妮，但安妮的婚姻状况未知，因此我们不能确定杰克是否在看一个未婚的人。然而，安妮在看乔治，而乔治是未婚的。由于安妮的婚姻状况未知，我们不能确定安妮是否已婚，因此不能直接得出安妮是否在看一个未婚的人。

但是，题目明确指出杰克是已婚的，并且杰克在看安妮，而安妮在看乔治，乔治是未婚的。因此，可以确定的是，已婚的杰克在看安妮，而安妮又在看未婚的乔治。虽然我们不能直接确定杰克是否在看一个未婚的人（因为安妮的婚姻状况未知），但根据题目描述，已婚的杰克在看安妮，而安妮的行为不影响杰克是否在看一个未婚的人这一结论。

因此，可以肯定的是，已婚的杰克在看安妮，而安妮的行为（看乔治）不影响杰克是否在看一个未婚的人这一结论。所以，有已婚的人（杰克）在看一个可能的人（安妮，但安妮的婚姻状况不影响结论），而安妮的行为（看乔治）表明有已婚的人（安妮可能是已婚的，但未确定）在看一个未婚的人（乔治）。

综上所述，可以确定的是，有已婚的人（杰克）在看一个可能的人（安妮，但安妮的婚姻状况不影响结论），而安妮的行为（看乔治）表明有已婚的人（安妮可能是已婚的，但未确定）在看一个未婚的人（乔治）。因此，可以肯定地说，有已婚的人在看未婚的人。


In [37]:
# 系统提示
SYSTEM_PROMPT = """你是一个精确的逻辑推理专家。在回答问题时，请：
1. 仔细分析所有给定条件
2. 考虑所有可能的情况（穷举法）
3. 对每种可能性进行逐一验证
4. 得出明确的结论
5. 确保推理过程严密无误，不遗漏任何可能性"""

# 提示
PROMPT = "杰克在看安妮。安妮在看乔治。杰克已婚，乔治未婚，我们不知道安妮是否已婚。有已婚的人在看未婚的人吗？"

# 打印模型的响应
print(get_completion(PROMPT, SYSTEM_PROMPT))

为了回答这个问题，我们需要仔细分析给定的信息并考虑所有可能的情况。

已知信息：
1. 杰克在看安妮。
2. 安妮在看乔治。
3. 杰克已婚。
4. 乔治未婚。
5. 安妮的婚姻状况未知。

我们需要确定是否有已婚的人在看未婚的人。

### 分析情况

#### 情况1：安妮已婚
- 杰克已婚，且杰克在看安妮（安妮已婚）。
- 安妮在看乔治（乔治未婚）。
- 结论：安妮（已婚）在看乔治（未婚）。

#### 情况2：安妮未婚
- 杰克已婚，且杰克在看安妮（安妮未婚）。
- 安妮在看乔治（乔治未婚）。
- 结论：杰克（已婚）在看安妮（未婚）。

### 总结
无论安妮是否已婚，都有已婚的人在看未婚的人：
- 如果安妮已婚，那么安妮（已婚）在看乔治（未婚）。
- 如果安妮未婚，那么杰克（已婚）在看安妮（未婚）。

因此，最终结论是：有已婚的人在看未婚的人。
